In [ ]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Aula Interativa 1 - Cientista de Dados - Apache Spark") \
    .getOrCreate()

spark.version


In [ ]:
# Carregando a planilha CSV
titanic_df = spark.read.csv('C:/Users/pedro.guerra/Downloads/XPE/cientista-dados/aula1/titanic-partial.csv',header='True',inferSchema='True')

# Detalhes dos atributos em https://www.kaggle.com/c/titanic/data?select=train.csv
titanic_df.printSchema()


In [ ]:
# Carregando o arquivo JSON

survived_df = spark.read.json("C:/Users/pedro.guerra/Downloads/XPE/cientista-dados/aula1/titanic-survived.json")
survived_df.printSchema()
survived_df.show()

In [ ]:
# Juntando os dois dataframes.

titanic_df = titanic_df.join(survived_df, titanic_df.PassengerId == survived_df.PassengerId)

titanic_df.printSchema()


In [ ]:
titanic_df.count()

In [ ]:
titanic_df.show(5)

In [ ]:
titanic_df.select("Survived", "Name", "Pclass","Embarked").show()

In [ ]:
titanic_df.describe().show()

In [ ]:
titanic_df.describe('Age', 'Fare').show()

In [ ]:
titanic_df.select('Age', 'Fare').summary().show()

In [ ]:
titanic_df.groupBy("Survived").count().show()


In [ ]:
titanic_df.groupBy("Sex","Survived").count().show()

In [ ]:
titanic_df.groupBy("Pclass","Survived").count().show()



In [ ]:
from pyspark.sql.functions import col, regexp_extract


titanic_df = titanic_df.withColumn("Initial", regexp_extract(col("Name"),"([A-Za-z]+)\.",1))

titanic_df.show()

In [ ]:
titanic_df.select("Initial").distinct().show()


In [ ]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [ ]:
titanic_df.select("Initial").distinct().show()

In [ ]:
titanic_df.groupby('Initial').avg('Age').collect()


In [ ]:
from pyspark.sql.functions import col

titanic_df.filter(col("Initial") == 'Miss').select("Name", "Age", "Initial").show(5)

titanic_df.filter(titanic_df.Initial == 'Miss').select("Name", "Age", "Initial").show(5)

In [ ]:
from pyspark.sql.functions import when

titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Miss") & (titanic_df["Age"].isNull()), 22).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Other") & (titanic_df["Age"].isNull()), 46).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Master") & (titanic_df["Age"].isNull()), 5).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mr") & (titanic_df["Age"].isNull()), 33).otherwise(titanic_df["Age"]))
titanic_df = titanic_df.withColumn("Age",when((titanic_df["Initial"] == "Mrs") & (titanic_df["Age"].isNull()), 36).otherwise(titanic_df["Age"]))


In [ ]:
titanic_df.groupBy("Embarked").count().show()


In [ ]:
titanic_df = titanic_df.na.fill({"Embarked" : 'S'})

In [ ]:
titanic_df.groupBy("Embarked").count().show()

In [ ]:
titanic_df.describe("Cabin").show()

In [ ]:
titanic_df = titanic_df.drop("Cabin")

titanic_df.printSchema()


In [ ]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [ ]:
from pyspark.sql.functions import col


titanic_df.groupBy("Family_Size").count().show()


In [ ]:
from pyspark.sql.functions import asc

titanic_df.select("Name", "Family_Size").orderBy(col("Family_Size").desc()).show()


In [ ]:
from pyspark.sql.functions import lit

titanic_df = titanic_df.withColumn('Alone',lit(0))


In [ ]:
from pyspark.sql.functions import lit


titanic_df = titanic_df.withColumn('Alone',lit(0))
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(titanic_df["Alone"]))



In [ ]:
titanic_df.columns


In [ ]:
spark.conf.set("spark.sql.codegen.wholeStage", False)

titanic_df.filter(titanic_df.Age > 70).show()


In [ ]:
from pyspark.sql.functions import desc


titanic_df.orderBy(desc("age")).show(1)


In [ ]:
from pyspark.sql.functions import asc

titanic_df.orderBy(asc("age")).show(1)

In [ ]:
titanic_df.groupBy('survived').agg({'age': 'avg'}).show()

In [ ]:
titanic_df.stat.corr("age", "fare")


In [ ]:
titanic_df.stat.corr("age", "family_size")


In [ ]:
# https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.DataFrameStatFunctions.html#pyspark.sql.DataFrameStatFunctions
titanic_df.stat.crosstab("Embarked", "PClass").show()

In [ ]:
titanic_df.groupBy('pclass').agg({'fare': 'avg'}).show()

In [ ]:
# UDFs ajudam plugar funções complexas
# extrair o last Name

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf


def getLastName(str):
    return str.split(",")[0]

getLastNameUDF = udf(lambda z:getLastName(z),StringType())   

titanic_df = titanic_df.withColumn("lastName", getLastNameUDF(titanic_df.Name))

In [ ]:
titanic_df.select("lastName").distinct().show()

In [ ]:
titanic_df.select("lastName").distinct().count()

In [ ]:
titanic_df.groupBy("lastName").count().show()